In [24]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
import numpy as np

In [25]:
# Tensorflow log verbosity: DEBUG, INFO, WARN, ERROR, and FATAL
tf.logging.set_verbosity(tf.logging.INFO)
# Tensorflow version
print(tf.__version__)

0.12.head


In [15]:
# Create 100 phony x, y data points in NumPy, y = x * 0.1 + 0.3
x_data = np.random.rand(100).astype(np.float32)
y_data = x_data * 0.1 + 0.3
# Try to find values for W and b that compute y_data = W * x_data + b
# (We know that W should be 0.1 and b 0.3, but TensorFlow will
# figure that out for us.)
W = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b = tf.Variable(tf.zeros([1]))
y = W * x_data + b

In [5]:
# Minimize the mean squared errors.
loss = tf.reduce_mean(tf.square(y - y_data))
optimizer = tf.train.GradientDescentOptimizer(0.5)
train_op = optimizer.minimize(loss)
# Before starting, initialize the variables.  We will 'run' this first.
init_op = tf.global_variables_initializer()

In [6]:
# Launch the graph.
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                        log_device_placement=True))

In [0]:
sess.run(init_op)
# Fit the line.
for step in range(201):
    sess.run(train_op)
    if step % 20 == 0:
      print("#{0}:  W={1}: b={2}".format(step, sess.run(W), sess.run(b)))
      

In [8]:
sess.close()

In [9]:
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
output = tf.mul(input1, input2)

with tf.Session() as sess:
  print(sess.run([output], feed_dict={input1:[7.], input2:[2.]}))

[array([ 14.], dtype=float32)]


In [39]:
#
# WARNING:  Disable GPU version 'sudo prime-select intel'  and restart kernel.
# Otherwise: Error Message 'InvalidArgumentError (see above for traceback):
# WhereOp: Race condition between counting ...'
#
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.contrib.learn.python.learn.metric_spec import MetricSpec

import tensorflow as tf
import numpy as np

In [40]:
# Tensorflow log verbosity: DEBUG, INFO, WARN, ERROR, and FATAL
tf.logging.set_verbosity(tf.logging.INFO)

In [41]:
gIrisDir = "/home/data/asarcar-data/tmp/IRIS"
gIrisModelDir = gIrisDir + "/model"
gIrisDataDir = gIrisDir + "/data"
# Data sets
IRIS_TRAINING = gIrisDataDir + "/iris_training.csv"
IRIS_TEST = gIrisDataDir + "/iris_test.csv"

In [42]:
# Load datasets.
training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TRAINING,
    target_dtype=np.int,
  features_dtype=np.float32)
test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TEST,
    target_dtype=np.int,
    features_dtype=np.float32)

In [0]:
print(type(test_set))
print(test_set.data)
print(type(test_set.target))

In [0]:
# validation_metrics = {"accuracy": tf.contrib.metrics.streaming_accuracy,
#                      "precision": tf.contrib.metrics.streaming_precision,
#                       "recall": tf.contrib.metrics.streaming_recall}

# Evaluate every 50 steps against test data to track
# how well the model is generalizing
validation_metrics = {
  "accuracy" : MetricSpec(
    metric_fn=tf.contrib.metrics.streaming_accuracy,
    prediction_key="classes"),
  "precision" : MetricSpec(
    metric_fn=tf.contrib.metrics.streaming_precision,
    prediction_key="classes"),
  "recall" : MetricSpec(
    metric_fn=tf.contrib.metrics.streaming_recall,
    prediction_key="classes")
}

validation_monitor = tf.contrib.learn.monitors.ValidationMonitor(
  test_set.data,
  test_set.target,
  every_n_steps=50,
  metrics=validation_metrics,
  early_stopping_metric="loss",
  early_stopping_metric_minimize=True,
  early_stopping_rounds=200)

In [0]:
# Specify that all features and data type ( real-value) data
# Features:
#   Sepal Length, Sepal Width, Petal Length, and Petal Width
# Target:
#   Species of the IRIS: 0 - Iris Setosa, 1 Iris Versicolor, or 2 - Iris Virginica
feature_columns = [tf.contrib.layers.real_valued_column("", dimension=4)]
# ValidationMonitors rely on saved checkpoints to perform evaluation.
# Checkpoint every 1 sec
# Build 3 layer DNN with 10, 20, 10 units respectively.
classifier = tf.contrib.learn.DNNClassifier(feature_columns=feature_columns,
                                            hidden_units=[10, 20, 10],
                                            n_classes=3,
                                            model_dir=gIrisModelDir,
                                            config=tf.contrib.learn.RunConfig(save_checkpoints_secs=1))
# Fit model: attaching monitors
classifier.fit(x=training_set.data,
               y=training_set.target,
               steps=10000,
               monitors=[validation_monitor])

# Evaluate accuracy.
accuracy_score = classifier.evaluate(x=test_set.data,
				     y=test_set.target)["accuracy"]
print('Accuracy: {0:f}'.format(accuracy_score))

In [0]:
# Classify two new flower samples.
new_samples = np.array(
    [[6.4, 3.2, 4.5, 1.5], [5.8, 3.1, 5.0, 1.7]], dtype=float)
y = list(classifier.predict(new_samples, as_iterable=True))
print('Predictions: {}'.format(str(y)))

In [28]:
#
# WARNING:  Disable GPU version 'sudo prime-select intel'  and restart kernel.
# Otherwise: Error Message 'InvalidArgumentError (see above for traceback):
# WhereOp: Race condit
#
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from tensorflow.contrib.learn.python.learn.metric_spec import MetricSpec
import itertools
import numpy as np
import pandas as pd
import tensorflow as tf


In [29]:
# Tensorflow log verbosity: DEBUG, INFO, WARN, ERROR, and FATAL
tf.logging.set_verbosity(tf.logging.INFO)

In [30]:
gIrisDir = "/home/data/asarcar-data/tmp/IRIS"
gIrisModelDir = gIrisDir + "/model"
gIrisDataDir = gIrisDir + "/data"
gTrainSteps = 2000
# Data sets
IRIS_TRAINING = gIrisDataDir + "/iris_training.csv"
IRIS_TEST = gIrisDataDir + "/iris_test.csv"
IRIS_PREDICTION = gIrisDataDir + "/iris_prediction.csv"
#
# HEADER FORMAT
#   num_rows, num_features, target_classes
#   120,4,setosa,versicolor,virginica
# DATA FORMAT
#   Sepal Length (1), Sepal Width(2), Petal Length(3), Petal Width(4), Species(5)
#     Species: 0 - Iris Setosa, 1 Iris Versicolor, or 2 - Iris Virginica
#   6.4,2.8,5.6,2.2,2
#
COLUMNS = ["sepal_length", "sepal_width", "petal_length", "petal_width", "species"]
FEATURES = ["sepal_length", "sepal_width", "petal_length", "petal_width"]
LABEL = "species"
gNumClasses = 3

In [31]:
# Load datasets.
training_set = pd.read_csv(IRIS_TRAINING, skipinitialspace=True,
			   skiprows=1,names=COLUMNS)
test_set = pd.read_csv(IRIS_TEST, skipinitialspace=True,
		       skiprows=1,names=COLUMNS)
prediction_set = pd.read_csv(IRIS_PREDICTION, skipinitialspace=True,
			     skiprows=1,names=COLUMNS)


In [32]:
def input_fn(df):
  # Creates a dictionary mapping from each continuous feature column name (k) to
  # the values of that column stored in a constant Tensor.
  feature_cols = {k: tf.constant(df[k].values) for k in COLUMNS}
  # Converts the label column into a constant Tensor.
  label = tf.constant(df[LABEL].values)
  # Returns the feature columns and the label.
  return feature_cols, label  

In [0]:
# Specify that all features and data type ( real-value) data
feature_cols = [tf.contrib.layers.real_valued_column(k) for k in FEATURES]
# Build 3 layer DNN with 10, 20, 10 units respectively.
classifier = tf.contrib.learn.DNNClassifier(feature_columns=feature_cols,
                                            hidden_units=[10, 20, 10],
                                            n_classes=gNumClasses)

In [0]:
# Fit model: attaching monitors
classifier.fit(input_fn = lambda: input_fn(training_set), steps=gTrainSteps)

In [0]:
# Evaluate accuracy.
eval_result = classifier.evaluate(input_fn = lambda: input_fn(test_set), steps=1)
print('Accuracy: {0:f}'.format(eval_result["accuracy"]))

In [0]:
def predict_input_fn():
  # Bug in Predict Code when passing function as input_fn
  new_samples = {
    "sepal_length": tf.constant(np.array([6.4,5.8], dtype=np.float64)),
    "sepal_width": tf.constant(np.array([3.2,3.1], dtype=np.float64)),
    "petal_length": tf.constant(np.array([4.5,5.0], dtype=np.float64)),
    "petal_width": tf.constant(np.array([1.5,1.7], dtype=np.float64))
  }
  # FEATURES = ["sepal_length", "sepal_width", "petal_length", "petal_width"]
  new_result = tf.constant(np.array([1,2], dtype=np.int64))
  return new_samples, new_result
es = classifier.predict(input_fn = predict_input_fn)
predictions = list(itertools.islice(res, 2))
print ("Predictions: {}".format(str(predictions)))

In [11]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import itertools
import pandas as pd
import tensorflow as tf

In [2]:
tf.logging.set_verbosity(tf.logging.INFO)

In [3]:
# Data Format
# FEATURES:
# ----------------
# CRIM	Crime rate per capita
# ZN	Fraction of residential land zoned to permit 25,000+ sq ft lots
# INDUS	Fraction of land that is non-retail business
# NOX	Concentration of nitric oxides in parts per 10 million
# RM	Average Rooms per dwelling
# AGE	Fraction of owner-occupied residences built before 1940
# DIS	Distance to Boston-area employment centers
# TAX	Property tax rate per $10,000
# PTRATIO	Student-teacher ratio
#
# LABEL:
# ----------
# MEDV Median value of owner-occupied residences in $1,000s
COLUMNS = ["crim", "zn", "indus", "nox", "rm", "age",
           "dis", "tax", "ptratio", "medv"]
FEATURES = ["crim", "zn", "indus", "nox", "rm",
            "age", "dis", "tax", "ptratio"]
LABEL = "medv"
gUCIdir = "/home/data/asarcar-data/tmp/UCI/"
gUCIdata = gUCIdir + "data/"
gUCImodel = gUCIdir + "model/"
gTrainFileName = gUCIdata + "boston_train.csv"
gTestFileName = gUCIdata + "boston_test.csv"
gTrainSteps = 5000


In [4]:
training_set = pd.read_csv(gTrainFileName, skipinitialspace=True,skiprows=1, names=COLUMNS)
test_set = pd.read_csv(gTestFileName, skipinitialspace=True,skiprows=1, names=COLUMNS)

In [5]:
feature_cols = [tf.contrib.layers.real_valued_column(k) for k in FEATURES]

In [0]:
regressor = tf.contrib.learn.DNNRegressor(
  feature_columns = feature_cols, hidden_units=[10,10])

In [7]:
# Build Input Function for Transforming Raw Data and Feeding to DNN
def input_fn(data_set):
  feature_cols = {k: tf.constant(data_set[k].values) for k in FEATURES}
  labels = tf.constant(data_set[LABEL].values)
  return feature_cols, labels

In [0]:
regressor.fit(input_fn=lambda:input_fn(training_set),steps=gTrainSteps)

In [0]:
ev = regressor.evaluate(input_fn=lambda:input_fn(test_set),steps=1)
loss_score = ev["loss"]
print("Loss: {0:f}".format(loss_score))

In [0]:
y = regressor.predict(input_fn=lambda: input_fn(predict_set))
# .predict returns an iterator => convert to list
predictions = list(itertools.islice(y, 6))
print ("Predictions: {}".format(str(predictions)))

In [1]:
#
# WARNING: Disable GPU version 'sudo prime-select intel'  and restart kernel.
# Otherwise: Error Message InvalidArgumentError: WhereOp: Race condition
# between counting the number of true elements and writing them.
# When counting, saw xxxx elements; but when writing their indices,
# saw xx elements.
#
# Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Example code for TensorFlow Wide & Deep Tutorial using TF.Learn API."""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from six.moves import urllib

import pandas as pd
import tensorflow as tf

In [2]:
# Tensorflow log verbosity: DEBUG, INFO, WARN, ERROR, and FATAL
tf.logging.set_verbosity(tf.logging.INFO)

In [3]:
# Parameter Globals
gCensusDir = "/home/data/asarcar-data/tmp/CENSUS/"
# 'Initial learning rate.'
gInputDataDir           =  gCensusDir + "data/"
# "Base directory for output models."
gModelDir = gCensusDir + "model/"
# Model Types: "wide",  "deep",  "wide_n_deep"
gModelType = "wide_n_deep"
# "Number of training steps."
gTrainSteps = 200
# URL Base where data directory
gURLBase = "http://mlr.cs.umass.edu/ml/machine-learning-databases/adult/"
# TrainFile
gTrainFile = "adult.data"
# TestFile
gTestFile = "adult.test"
# Train File URL
gTrainFileURL = gURLBase + gTrainFile
# Test File URL
gTestFileURL = gURLBase + gTestFile
# Train FileName
gTrainFileName = gInputDataDir + gTrainFile
# Test FileName
gTestFileName = gInputDataDir + gTestFile


In [4]:
# CENSUS DATA FORMAT
COLUMNS = ["age", "workclass", "fnlwgt", "education", "education_num",
           "marital_status", "occupation", "relationship", "race", "gender",
           "capital_gain", "capital_loss", "hours_per_week", "native_country",
           "income_bracket"]
LABEL_COLUMN = "label"
CATEGORICAL_COLUMNS = ["workclass", "education", "marital_status",
		       "occupation","relationship", "race", "gender",
		       "native_country"]
CONTINUOUS_COLUMNS = ["age", "education_num", "capital_gain",
		      "capital_loss","hours_per_week"]

In [5]:
def maybe_download():
  """Maybe downloads training data and returns train and test file names."""
  # pylint: disable=line-too-long
  urllib.request.urlretrieve(gTrainFileURL, gTrainFileName)  
  print("Training data is downloaded to %s" % gTrainFileName)
  urllib.request.urlretrieve(gTestFileURL, gTestFileName)  
  print("Test data is downloaded to %s" % gTestFileName)
  return gTrainFileName, gTestFileName

In [6]:
def build_estimator(model_dir):
  """Build an estimator."""
  # Sparse base columns.
  gender = tf.contrib.layers.sparse_column_with_keys(column_name="gender",
                                                     keys=["female", "male"])
  education = tf.contrib.layers.sparse_column_with_hash_bucket(
      "education", hash_bucket_size=1000)
  relationship = tf.contrib.layers.sparse_column_with_hash_bucket(
      "relationship", hash_bucket_size=100)
  workclass = tf.contrib.layers.sparse_column_with_hash_bucket(
      "workclass", hash_bucket_size=100)
  occupation = tf.contrib.layers.sparse_column_with_hash_bucket(
      "occupation", hash_bucket_size=1000)
  native_country = tf.contrib.layers.sparse_column_with_hash_bucket(
      "native_country", hash_bucket_size=1000)
  # Continuous base columns.
  age = tf.contrib.layers.real_valued_column("age")
  education_num = tf.contrib.layers.real_valued_column("education_num")
  capital_gain = tf.contrib.layers.real_valued_column("capital_gain")
  capital_loss = tf.contrib.layers.real_valued_column("capital_loss")
  hours_per_week = tf.contrib.layers.real_valued_column("hours_per_week")

  # Transformations.
  age_buckets = tf.contrib.layers.bucketized_column(age,
                                                    boundaries=[
                                                        18, 25, 30, 35, 40, 45,
                                                        50, 55, 60, 65
                                                    ])
  # L1 regularization tends to make model weights stay at zero,
  # creating sparser models, whereas L2 regularization also tries
  # to make the model weights closer to zero but not necessarily zero.
  # Strong L1 regularization => smaller model size:
  # Desirable when feature space is large & sparse or resource constraints
  opt = tf.train.FtrlOptimizer(
    learning_rate=0.1,
    l1_regularization_strength=1.0,
    l2_regularization_strength=1.0)
  # Wide columns and deep columns.
  wide_columns = [gender, native_country, education, occupation, workclass,
                  relationship, age_buckets,
                  tf.contrib.layers.crossed_column([education, occupation],
                                                   hash_bucket_size=int(1e4)),
                  tf.contrib.layers.crossed_column(
                      [age_buckets, education, occupation],
                      hash_bucket_size=int(1e6)),
                  tf.contrib.layers.crossed_column([native_country, occupation],
                                                   hash_bucket_size=int(1e4))]
  deep_columns = [
      tf.contrib.layers.embedding_column(workclass, dimension=8),
      tf.contrib.layers.embedding_column(education, dimension=8),
      tf.contrib.layers.embedding_column(gender, dimension=8),
      tf.contrib.layers.embedding_column(relationship, dimension=8),
      tf.contrib.layers.embedding_column(native_country,
                                         dimension=8),
      tf.contrib.layers.embedding_column(occupation, dimension=8),
      age,
      education_num,
      capital_gain,
      capital_loss,
      hours_per_week,
  ]

  if gModelType == "wide":
    m = tf.contrib.learn.LinearClassifier(model_dir=model_dir,
					  feature_columns=wide_columns,
					  optimizer = opt)
  elif gModelType == "deep":
    m = tf.contrib.learn.DNNClassifier(model_dir=model_dir,
                                       feature_columns=deep_columns,
                                       hidden_units=[100, 50])
  else:
    m = tf.contrib.learn.DNNLinearCombinedClassifier(
        model_dir=model_dir,
        linear_feature_columns=wide_columns,
        dnn_feature_columns=deep_columns,
        dnn_hidden_units=[100, 50])
  return m

In [7]:
def input_fn(df):
  """Input builder function."""
  # Creates a dictionary mapping from each continuous feature column name (k) to
  # the values of that column stored in a constant Tensor.
  continuous_cols = {k: tf.constant(df[k].values) for k in CONTINUOUS_COLUMNS}
  # Creates a dictionary mapping from each categorical feature column name (k)
  # to the values of that column stored in a tf.SparseTensor.
  categorical_cols = {
      k: tf.SparseTensor(
          indices=[[i, 0] for i in range(df[k].size)],
          values=df[k].values,
          shape=[df[k].size, 1])
      for k in CATEGORICAL_COLUMNS}
  # Merges the two dictionaries into one.
  feature_cols = dict(continuous_cols)
  feature_cols.update(categorical_cols)
  # Converts the label column into a constant Tensor.
  label = tf.constant(df[LABEL_COLUMN].values)
  # Returns the feature columns and the label.
  return feature_cols, label

In [8]:
def train_and_eval():
  """Train and evaluate the model."""
  train_file_name, test_file_name = maybe_download()
  df_train = pd.read_csv(
      tf.gfile.Open(train_file_name),
      names=COLUMNS,
      skipinitialspace=True,
      engine="python")
  df_test = pd.read_csv(
      tf.gfile.Open(test_file_name),
      names=COLUMNS,
      skipinitialspace=True,
      skiprows=1,
      engine="python")
  # remove NaN elements
  df_train = df_train.dropna(how='any', axis=0)
  df_test = df_test.dropna(how='any', axis=0)

  df_train[LABEL_COLUMN] = (
      df_train["income_bracket"].apply(lambda x: ">50K" in x)).astype(int)
  df_test[LABEL_COLUMN] = (
      df_test["income_bracket"].apply(lambda x: ">50K" in x)).astype(int)

  model_dir = gModelDir
  print("model directory = %s" % model_dir)

  m = build_estimator(model_dir)
  m.fit(input_fn=lambda: input_fn(df_train), steps=gTrainSteps)
  results = m.evaluate(input_fn=lambda: input_fn(df_test), steps=1)
  for key in sorted(results):
    print("%s: %s" % (key, results[key]))
  print('Evaluated Accuracy: {0:f}'.format(results["accuracy"]))

In [0]:
train_and_eval()

In [45]:
#  Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
"""DNNRegressor with custom estimator for abalone dataset."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

In [47]:
tf.logging.set_verbosity(tf.logging.INFO)

# Learning rate for the model
LEARNING_RATE = 0.001

In [62]:
gAbaloneDir = "/home/data/asarcar-data/tmp/ABALONE"
gAbaloneModelDir = gAbaloneDir + "/model"
gAbaloneDataDir = gAbaloneDir + "/data"
gTrainSteps = 5000
# Data sets
gAbaloneTrainFileName = gAbaloneDataDir + "/abalone_train.csv"
gAbaloneTestFileName = gAbaloneDataDir + "/abalone_test.csv"
gAbalonePredictFileName = gAbaloneDataDir + "/abalone_predict.csv"

In [49]:
# pylint: disable=unused-argument
def model_fn(features, targets, mode, params):
  """Model function for Estimator."""
  # Connect the first hidden layer to input layer
  # (features) with relu activation
  first_hidden_layer = tf.contrib.layers.relu(features, 10)

  # Connect the second hidden layer to first hidden layer with relu
  second_hidden_layer = tf.contrib.layers.relu(first_hidden_layer, 10)

  # Connect the output layer to second hidden layer (no activation fn)
  output_layer = tf.contrib.layers.linear(second_hidden_layer, 1)

  # Reshape output layer to 1-dim Tensor to return predictions
  predictions = tf.reshape(output_layer, [-1])
  predictions_dict = {"ages": predictions}

  # Calculate loss using mean squared error
  loss = tf.contrib.losses.mean_squared_error(predictions, targets)

  train_op = tf.contrib.layers.optimize_loss(
      loss=loss,
      global_step=tf.contrib.framework.get_global_step(),
      learning_rate=params["learning_rate"],
      optimizer="SGD")

  return predictions_dict, loss, train_op


In [63]:
def load_files():
  # Training examples
  train_set = tf.contrib.learn.datasets.base.load_csv_without_header(
      filename=gAbaloneTrainFileName,
      target_dtype=np.int,
      features_dtype=np.float64)
  # Test examples
  test_set = tf.contrib.learn.datasets.base.load_csv_without_header(
      filename=gAbaloneTestFileName,
      target_dtype=np.int,
      features_dtype=np.float64)

  # Set of 7 examples for which to predict abalone ages
  pred_set = tf.contrib.learn.datasets.base.load_csv_without_header(
      filename=gAbalonePredictFileName,
      target_dtype=np.int,
      features_dtype=np.float64)

  return train_set, test_set, pred_set

In [64]:
# Set model params
model_params = {"learning_rate": LEARNING_RATE}

In [0]:
training_set, test_set, prediction_set = load_files()

# Build 2 layer fully connected DNN with 10, 10 units respectively.
nn = tf.contrib.learn.Estimator(
  model_fn=model_fn, params=model_params)

In [0]:
# Fit
nn.fit(x=training_set.data, y=training_set.target, steps=gTrainSteps)

In [0]:
# Score accuracy
ev = nn.evaluate(x=test_set.data, y=test_set.target, steps=1)
loss_score = ev["loss"]
print("Loss: %s" % loss_score)

In [0]:
# Print out predictions
predictions = nn.predict(x=prediction_set.data,
                         as_iterable=True)
for i, p in enumerate(predictions):
  print("Prediction %s: %s" % (i + 1, p["ages"]))